In [1]:
Pkg.update() 
Pkg.add("JuMP")
Pkg.add("GLPKMathProgInterface")

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package JuMP is already installed
INFO: Package GLPKMathProgInterface is already installed


In [2]:
using JuMP
using MathProgBase
using GLPKMathProgInterface

In [5]:
include("util.jl")
include("network.jl")

In [6]:
# read in neural networks
small_nnet = read_nnet("small_nnet.txt")
large_nnet = read_nnet("ACASXU_nnet_1.txt")

Network(Layer[Layer([0.22763, -0.188762, 0.0534094, -0.377861, -0.0812531, -0.588651, 0.0746065, -0.392887, -0.356303, -0.167712  …  0.221606, -0.00650873, -0.634843, -0.153833, -0.0573027, -0.109229, 0.0370344, 0.0538087, -0.44755, 0.175917], [0.0540062 0.0540062 … -0.180027 0.242194; -1.12374 -1.12374 … -0.00917929 0.055623; … ; -1.57631 -1.57631 … -0.00832176 -0.0926607; 0.620367 0.620367 … -0.0146141 0.262575]), Layer([0.08355, -0.0213261, 0.756291, -1.25082, 0.178469, -0.339158, 0.109226, -1.26214, -0.356488, -0.00829379  …  0.655079, 1.25241, 0.0390987, -0.0242382, -0.385696, -0.359071, -0.155856, 0.0285971, -0.426623, 0.780925], [-0.184202 -0.184202 … 0.337388 0.395671; 0.0171639 0.0171639 … -0.0371157 0.0196442; … ; -1.24508 -1.24508 … 0.646698 -0.159838; -0.348713 -0.348713 … -0.60238 -0.142155]), Layer([-0.408486, 1.06875, -0.101351, 0.291739, 0.361906, -1.17598, -1.08494, -0.525396, -0.485121, -0.563451  …  -0.563058, -0.217101, 0.570546, -0.523408, -1.49997, -0.261921, -0.5

In [22]:
function add_input_constraints(nnet::Network, inputs::Matrix{Float64}, m::Model) # change inputs to vector here as well
    n_inputs = size(nnet.layers[1].weights)[2]
    @variable(m, x_in[1:n_inputs])
    @constraint(m, constr_in[i=1:n_inputs], x_in[i] == inputs[i])    
end

add_input_constraints (generic function with 1 method)

In [23]:
# test add input constraints
inputs = [1.0 2.0 3.0 4.0 5.0] # change inputs to set of vectors - type Ax <= b
m = Model(solver = GLPKSolverLP())
add_input_constraints(large_nnet, inputs, m)

5-element Array{JuMP.ConstraintRef,1}:
 x_in[1] = 1
 x_in[2] = 2
 x_in[3] = 3
 x_in[4] = 4
 x_in[5] = 5

In [36]:
function add_output_constraints(nnet::Network, outputs::Matrix{Float64}, m::Model)
    n_layers = length(nnet.layers)
    n_outputs = size(nnet.layers[n_layers].weights)[1]
    @variable(m, x_out[1:n_outputs])
    @constraint(m, constr_out[i=1:n_outputs], x_out[i] == outputs[i])
end

add_output_constraints (generic function with 1 method)

In [37]:
# test add_output_constraints - Ax = b
outputs = [5.0 4.0 3.0 2.0 1.0]
m = Model(solver = GLPKSolverLP())
add_output_constraints(large_nnet, outputs, m)

5-element Array{JuMP.ConstraintRef,1}:
 x_out[1] = 5
 x_out[2] = 4
 x_out[3] = 3
 x_out[4] = 2
 x_out[5] = 1

In [ ]:
function add_network_constraints(nnet::Network, m::Model)
    M = 1 # is this a suitable ("sufficiently large") constant?
    n_layers = length(nnet.layer_sizes)
    @variable(m, x_net[i=1:n_layers, j=1:nnet.layer_sizes[i]])
    @variable(m, delta[i=1:n_layers, j=1:nnet.layer_sizes[i]], Bin)
    # 3 dimensional array of constraints - 4 for each node
    for (l, layer) in enumerate(nnet.layers)
        
        #@constraint(m, x_n, x[i] >= ((x_n * layer.weights) + layer.biases)[i])
        #@constraint(m, x_n, x[i] <= ((x_n * layer.weights) + layer.biases)[i] + M*delta[i])
        @constraint(m, x[i] >= 0)
        @constraint(m, x_n, x[i] <= M*(1 - delta[i]))  
    end
end

In [ ]:
add_network_constraints(small_nnet, m)

In [ ]:
function initialize_constraints(inputs::Matrix{Float64}, outputs::Matrix{Float64},
                                network::Network, m::Model)
    add_input_constraints(inputs, m)
    #add_output_constraints(outputs, m)
    #add_network_constraints(network, m)
end

In [ ]:
# Setup our model and add our contraints
model = Model(solver = GLPKSolverLP()) # use GLPK LP as our solver
initialize_constraints(inputs, outputs, network, model)

# Solve the model
status = solve(model)

# Print out the solution
print(status)